![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/logo_MCV_W.png)

# **Colorize Black and White Photos using a Caffe Model in OpenCV**

**In this lesson we'll learn how to use pre-trained models to automatically Colorize a Black and White (grayscale) Photo**


### **Colorizing black and white images is an amazingly useful and incredible technique achieved by deep learning.** 

[Colorful Image Colorization ](http://arxiv.org/pdf/1603.08511.pdf) 

- The authors embrace the underlying uncertainty of the problem (black and white to color conversion) by posing it as a classification task and use class-rebalancing at training time to increase the diversity of colors in the result. 
- The system is implemented as a feed-forward pass in a CNN at test time and is trained on over a million color images. 
- They evaluate our algorithm using a “colorization Turing test,” asking human participants to choose between a generated and ground truth color image. 
- Their method successfully fools humans on 32% of the trials, significantly higher than previous methods.

![](http://richzhang.github.io/colorization/resources/images/teaser3.jpg)

by Richard Zhang, Phillip Isola, Alexei A. Efros. In ECCV, 2016.

We'll be using the following Caffe model files that we'll download in the next cell below. These will be then loaded into OpenCV:

1. colorization_deploy_v2.prototext
2. colorization_release_v2.caffe
3. pts_in_hull.npy

In [1]:
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/colorize.zip
!unzip -qq colorize.zip

--2023-06-03 07:31:15--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/colorize.zip
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 3.5.246.157
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|3.5.246.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120120413 (115M) [application/zip]
Saving to: ‘colorize.zip’

colorize.zip        100%[===================>] 114.56M  20.5MB/s    in 6.4s    

2023-06-03 07:31:22 (18.0 MB/s) - ‘colorize.zip’ saved [120120413/120120413]



In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt

def imshow(title = "Image", image = None, size = 10):
  h, w = image.shape[:2]
  aspect_ratio = w/h
  plt.figure(figsize=(size * aspect_ratio,size))
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title(title)
  plt.show()

In [3]:
file_path = "colorize/blackandwhite/"
blackandwhite_imgs = [f for f in listdir(file_path) if isfile(join(file_path, f))]
kernel = 'colorize/pts_in_hull.npy'

net = cv2.dnn.readNetFromCaffe("colorize/colorization_deploy_v2.prototxt", "colorize/colorization_release_v2.caffemodel")
print(net.getLayerNames())

('conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'conv1_2norm', 'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'conv2_2norm', 'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_3norm', 'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3', 'relu4_3', 'conv4_3norm', 'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'conv5_3norm', 'conv6_1', 'relu6_1', 'conv6_2', 'relu6_2', 'conv6_3', 'relu6_3', 'conv6_3norm', 'conv7_1', 'relu7_1', 'conv7_2', 'relu7_2', 'conv7_3', 'relu7_3', 'conv7_3norm', 'conv8_1', 'relu8_1', 'conv8_2', 'relu8_2', 'conv8_3', 'relu8_3', 'conv8_313', 'conv8_313_rh', 'class8_313_rh', 'class8_ab', 'Silence')


In [4]:
# load cluster centers
pts_in_hull = np.load(kernel) 
print(pts_in_hull.shape)

(313, 2)


In [5]:
# populate cluster centers as 1x1 convolution kernel
pts_in_hull = pts_in_hull.transpose().reshape(pts_in_hull.shape[1], pts_in_hull.shape[0], 1, 1)
print(pts_in_hull.shape)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull.astype(np.float32)]

# because in openCV, 'conv8_313_rh' is not filled by 2.606 even it need to be filled.
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]

(2, 313, 1, 1)


In [8]:
for image in blackandwhite_imgs:
  img = cv2.imread(file_path+image, cv2.IMREAD_GRAYSCALE)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
  img_rgb = (img_rgb/255.).astype(np.float32)
  img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2LAB)
        
  # pull out L channel
  img_l = img_lab[:,:,0]
        
  # get original image size
  (H_orig,W_orig) = img_rgb.shape[:2] 

  # resize image to network input size
  img_rs = cv2.resize(img_rgb, (224, 224)) 
        
  # resize image to network input size
  img_lab_rs = cv2.cvtColor(img_rs, cv2.COLOR_RGB2Lab)
  img_l_rs = img_lab_rs[:,:,0]
        
  # subtract 50 for mean-centering
  img_l_rs -= 50 

  net.setInput(cv2.dnn.blobFromImage(img_l_rs))
        
  # this is our result
  ab_dec = net.forward('class8_ab')[0,:,:,:].transpose((1,2,0)) 

  (H_out,W_out) = ab_dec.shape[:2]
  ab_dec_us = cv2.resize(ab_dec, (W_orig, H_orig))
  img_lab_out = np.concatenate((img_l[:,:,np.newaxis],ab_dec_us),axis=2) 
        
  # concatenate with original image L
  img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)

  # show original image
  imshow('Original', img)
  # Resize the corlized image to it's orginal dimensions 
  img_bgr_out = cv2.resize(img_bgr_out, (W_orig, H_orig), interpolation = cv2.INTER_AREA)
  imshow('Colorized', img_bgr_out)


Output hidden; open in https://colab.research.google.com to view.